In [ ]:
'''
Notebook dedicado à implementação e teste da AlexNet
https://www.youtube.com/watch?v=8GheVe2UmUM
./references/ImageNet Classification with Deep ConvolutionalNeural Networks.pdf

'''

In [ ]:
from pathlib import Path
import numpy as np
from sklearn.metrics import confusion_matrix
import itertools
import tensorflow as tf
import random
import glob
import shutil
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.metrics import *
import matplotlib.pyplot as plt


class AssetNotFoundError(OSError):
    pass


In [ ]:
'''Functions'''

# Função para exibir uma amostra das imagens que serão usadas
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 10, figsize=(25,25))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    '''
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    '''
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


In [ ]:
'''
Parâmetros
'''

steps = ['train', 'valid', 'test']
asset = 'VALE3'
classes = ['long', 'short', 'wait']
samples_sizes = [1150, 250, 245]

data_path = Path(f'./generated_data/{asset}')


In [ ]:
indicator = ' <- Insuficient images'
print('class: available / needed\n')
for classification in classes:
    search_str = str(data_path/f'*{classification}*')
    search_res = glob.glob(search_str)
    warning = ''
    if len(search_res) < sum(samples_sizes):
        warning = indicator
    print(f'{classification}: {str(len(search_res))} / {sum(samples_sizes)}{warning}')
    

In [ ]:
'''
Idendificação e configuração da GPU
'''

physical_devices = tf.config.experimental.list_physical_devices('GPU')
print('Num GPUs available: ', len(physical_devices))
# tf.config.experimental.set_memory_growth(physical_devices[0], True)


In [ ]:
'''
Cria as pastas de treinamento, validação e teste com as classes indicadas caso ainda não existam
'''

if not Path.is_dir(data_path):
    raise AssetNotFoundError('Asset folder not found.')

for step in steps:
    step_path = Path(data_path/step)
    step_path.mkdir(exist_ok=True)
    for classification in classes:
        classification_path = Path(step_path/classification)
        classification_path.mkdir(exist_ok=True)
print('Folder structure created.')


In [ ]:
'''
Seleciona aleatoriamente os gráficos que serão usados nas etapas de treinamento, velidação e teste e os move para as pastas correspondentes
As amostras são colhidas de forma equilibrada
'''

for i in range(0,len(steps)):
    step_path = Path(data_path/steps[i])
    for classification in classes:
        classification_path = Path(step_path/classification)
        search_str = str(data_path/f'*{classification}*')
        for chart in random.sample(glob.glob(search_str), samples_sizes[i]):
            shutil.move(chart, classification_path)
print('Images moved to destination folders.')


In [ ]:
rescale = 1/255
zoom_range = 0
target_size = (300,300)
batch_size = 100
class_mode = 'categorical'

train_datagen = ImageDataGenerator(rescale=rescale, zoom_range=zoom_range)
train_batches = train_datagen.flow_from_directory(f'{data_path}/train',
                                              target_size=target_size,
                                              batch_size=batch_size,
                                              class_mode=class_mode)

test_datagen = ImageDataGenerator(rescale=rescale, zoom_range=zoom_range)
test_batches = train_datagen.flow_from_directory(f'{data_path}/test',
                                              target_size=target_size,
                                              batch_size=batch_size,
                                              class_mode=class_mode)

valid_datagen = ImageDataGenerator(rescale=rescale, zoom_range=zoom_range)
valid_batches = train_datagen.flow_from_directory(f'{data_path}/valid',
                                              target_size=target_size,
                                              batch_size=batch_size,
                                              class_mode=class_mode)


In [ ]:
imgs, labels = next(train_batches)
plotImages(imgs)
print(labels[0:9])


In [ ]:
# AlexNet
# Conv2D
# MaxPool2D
# BatchNormalization
# Flatten
# Dense
# Dropout
model = Sequential([
    Conv2D(filters=4, kernel_size=5, strides=2, padding='same', activation='relu', input_shape=(target_size[0], target_size[1], 3)),
    BatchNormalization(),
    MaxPool2D(pool_size=4, strides=2, padding='same'),
    # Conv2D(filters=8, kernel_size=7, strides=4, padding='same', activation='relu'),
    # BatchNormalization(),
    # MaxPool2D(pool_size=4, strides=2, padding='same'),
    # Conv2D(filters=16, kernel_size=3, strides=1, padding='same', activation='relu'),
    # BatchNormalization(),
    # Conv2D(filters=16, kernel_size=3, strides=1, padding='same', activation='relu'),
    # BatchNormalization(),
    Conv2D(filters=8, kernel_size=3, strides=2, padding='same', activation='relu'),
    BatchNormalization(),
    MaxPool2D(pool_size=2, strides=1, padding='same'),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.25),
    Dense(200, activation='relu'),
    Dropout(0.25),
    Dense(100, activation='relu'),
    Dropout(0.25),
    Dense(3, activation='softmax')
])

# model.summary()


In [ ]:
model.compile(loss=categorical_crossentropy, optimizer='adam', metrics='accuracy')


In [ ]:
model.fit(x=train_batches, validation_data=valid_batches, epochs=8)


In [ ]:
test_imgs, test_labels = next(test_batches)
plotImages(test_imgs)
print(test_labels[0:9])


In [ ]:
predictions = model.predict(x=test_batches)


In [ ]:
test_batches.class_indices

In [ ]:
cm = confusion_matrix(y_true=test_batches.classes, y_pred=np.argmax(predictions, axis=-1))
cm_plot_labels = ['long', 'short', 'wait']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')
